In [1]:
#this imports the necessary modules
from music21 import *
import pickle, time, os
from sklearn.externals import joblib

In [2]:
#A is the path to the folder that holds all of the songs
# A = 'kern_files/wtc/'
A = corpus.getComposer('monteverdi') #ignore this for now


#these match the key or mode of the song with the appropriate label
tonal_major_dict = {(0, 0): 'CM', (1, 7): 'GM', (2, 2): 'DM', (3, 9): 'AM', (4, 4): 'EM', (5, 11): 'BM', (6, 6): 'F♯M',
               (-1, 5): 'FM', (-2, 10): 'B♭M', (-3, 3): 'E♭M', (-4, 8): 'A♭M', (-5, 1): 'D♭M', (-6, 6): 'G♭M'}
tonal_minor_dict = {(0, 9): 'am', (1, 4): 'em', (2, 11): 'bm', (3, 6): 'f♯m', (4, 1): 'c♯m', (5, 8): 'g♯m',
                    (6, 3): 'd♯m', (-1, 2): 'dm', (-2, 7): 'gm', (-3, 0): 'cm', (-4, 5): 'fm', (-5, 10): 'b♭m',
                    (-6, 3): 'e♭m'}
modal_major = [('f', 5), ('n', 0), ('f', 0), ('n', 7), ('f', 10), ('n', 5),
               (-1, 5), (0, 0), (-1, 0), (0, 7), (-1, 10), (0, 5)]
modal_minor = [('n', 2), ('n', 9), ('f', 2), ('f', 7), ('n', 4),
               (0, 2), (0, 9), (-1, 2), (-1, 7), (0, 4)]

In [3]:
# this function converts .krn or .xml files into the format we need
def data_converter(unparsed, modality, corpus_name):
    pitch_frequency_data = []
    all_parsed = []
    all_keys = []
    all_chords = []

    def parser(parsed_songs):
        for x in all_parsed:
            if type(x) is stream.Score:
    #             print('we have a score!')
                song_chords = []
                crd = x.chordify().flat.getElementsByClass(chord.Chord)
                if len(x.flat.getElementsByClass(key.KeySignature)) > 0:
                    key_sig = x.flat.getElementsByClass(key.KeySignature)[0].sharps
                    bass_note = crd[-1].bass().pitchClass
                    modal_type = []
                    if modality == 'tonal':
                        for c in tonal_major_dict.keys():
                            modal_type.append(c)
                        for c in tonal_minor_dict.keys():
                            modal_type.append(c)
                            #                 modal_type.append(c for c in tonal_minor_dict.keys())
                    elif modality == 'modal':
                        for c in modal_major:
                            modal_type.append(c)
                        for c in modal_minor:
                            modal_type.append(c)
                            #             print(modal_type, (key_sig, bass_note))
                    if (key_sig, bass_note) in modal_type:
                        print((key_sig, bass_note), 'is in there')
                        for c in crd:
                            song_chord = []
                            note_bass = (c.bass().pitchClass - bass_note) % 12
                            song_chord.append(note_bass)
                            song_chord.append(sorted(set([(n.pitchClass - bass_note - note_bass) % 12 for n in c])))
                            song_chords.append(song_chord)
                        all_chords.append(song_chords)
                        all_keys.append((key_sig, bass_note))
                    # this is for pitch class frequency data
                        all_notes = []
                        note_data = {}
                        song_percents = []
                        note_counter = 0
                        for xx in crd:
                            for yy in xx:
                                all_notes.append((yy.pitchClass - bass_note) % 12)
                                note_counter += 1
                        for xx in range(12):
                            note_data[xx] = 0
                        for xx in all_notes:
                            note_data[xx] += 1
                        for xx in range(12):
                            song_percents.append(note_data[xx] / len(all_notes) * 100)
                        pitch_frequency_data.append(song_percents)
                        print(len(pitch_frequency_data), len(all_keys))
                        print((key_sig, bass_note))
                    #                 all_corpus_labels.append((the_key[0].sharps, bass_note))

            elif type(x) is stream.Opus:
                print('we have an Opus!')
                for y in x:
                    song_chords = []
                    crd = y.chordify().flat.getElementsByClass(chord.Chord)
                    if len(x.flat.getElementsByClass(key.KeySignature)) > 0:
                        key_sig = y.flat.getElementsByClass(key.KeySignature)[0].sharps
                        bass_note = crd[-1].bass().pitchClass
                        modal_type = []
                        if modality == 'tonal':
                            for c in tonal_major_dict.keys():
                                modal_type.append(c)
                            for c in tonal_minor_dict.keys():
                                modal_type.append(c)
                                #                     modal_type.append(c for c in tonal_minor_dict.keys())
                        elif modality == 'modal':
                            for c in modal_major:
                                modal_type.append(c)
                            for c in modal_minor:
                                modal_type.append(c)
                        if (key_sig, bass_note) in modal_type:
                            print((key_sig, bass_note), 'is in there')
                            for c in crd:
                                song_chord = []
                                note_bass = (c.bass().pitchClass - bass_note) % 12
                                song_chord.append(note_bass)
                                song_chord.append(sorted(set([(n.pitchClass - bass_note - note_bass) % 12 for n in c])))
                                song_chords.append(song_chord)
                            all_chords.append(song_chords)
                            all_keys.append((key_sig, bass_note))
                        # this is for pitch class frequency data
                            all_notes = []
                            note_data = {}
                            song_percents = []
                            note_counter = 0
                            for xx in crd:
                                for yy in xx:
                                    all_notes.append((yy.pitchClass - bass_note) % 12)
                                    note_counter += 1
                            for xx in range(12):
                                note_data[xx] = 0
                            for xx in all_notes:
                                note_data[xx] += 1
                            for xx in range(12):
                                song_percents.append(note_data[xx] / len(all_notes) * 100)
                            pitch_frequency_data.append(song_percents)
                            print(len(pitch_frequency_data), len(all_keys))

    start = time.time()
    if type(unparsed) is str:

        for root, dirs, files in os.walk(unparsed):
            for file_name, x in zip(files, range(len(files))):
#                 print(file_name, '...', x + 1, 'of', len(files)+1, '...')
                if file_name.endswith('.krn'):
                    #                     print('it is a krn')
                    path = os.path.join(root, file_name)
                    st = time.time()
                    print('parsing', file_name)
                    all_parsed.append(converter.parse(path))
#                     parser(path)
                    fin = time.time()
                    print('parsed', len(all_parsed), 'of', len(unparsed), 'in', fin - st, 'seconds')
    else:
        for x in unparsed:
            st = time.time()
#             print('parsing', x)
            all_parsed.append(converter.parse(x))
#             parser(x)
            fin = time.time()
            print('parsed', len(all_parsed), 'of', len(unparsed), 'in', fin - st, 'seconds')
    print('entire process took:', time.time() - start, 'seconds')
    parser(all_parsed)
    joblib.dump((all_chords, all_keys), 'pickles/%s_continuo.pkl' % corpus_name, compress=9)
    joblib.dump((pitch_frequency_data, all_keys), 'pickles/%s_notes.pkl' % corpus_name, compress=9)

    print('ALL DONE!!! :) :) :)')
    
def chordify_maker(unparsed, modality, corpus_name):
    all_chords = []

    def parser(parsed_songs):
        for x in all_parsed:
            if type(x) is stream.Score:
    #             print('we have a score!')
                song_chords = []
                crd = x.chordify().flat.getElementsByClass(chord.Chord)
                if len(x.flat.getElementsByClass(key.KeySignature)) > 0:
                    key_sig = x.flat.getElementsByClass(key.KeySignature)[0].sharps
                    bass_note = crd[-1].bass().pitchClass
                    modal_type = []
                    if modality == 'tonal':
                        for c in tonal_major_dict.keys():
                            modal_type.append(c)
                        for c in tonal_minor_dict.keys():
                            modal_type.append(c)
                            #                 modal_type.append(c for c in tonal_minor_dict.keys())
                    elif modality == 'modal':
                        for c in modal_major:
                            modal_type.append(c)
                        for c in modal_minor:
                            modal_type.append(c)
                            #             print(modal_type, (key_sig, bass_note))
                    if (key_sig, bass_note) in modal_type:
                        print((key_sig, bass_note), 'is in there')
                        for c in crd:
                            song_chord = []
                            note_bass = (c.bass().pitchClass - bass_note) % 12
                            song_chord.append(note_bass)
                            song_chord.append(sorted(set([(n.pitchClass - bass_note - note_bass) % 12 for n in c])))
                            song_chords.append(song_chord)
                        all_chords.append(song_chords)
                        all_keys.append((key_sig, bass_note))
                    # this is for pitch class frequency data
                        all_notes = []
                        note_data = {}
                        song_percents = []
                        note_counter = 0
                        for xx in crd:
                            for yy in xx:
                                all_notes.append((yy.pitchClass - bass_note) % 12)
                                note_counter += 1
                        for xx in range(12):
                            note_data[xx] = 0
                        for xx in all_notes:
                            note_data[xx] += 1
                        for xx in range(12):
                            song_percents.append(note_data[xx] / len(all_notes) * 100)
                        pitch_frequency_data.append(song_percents)
                        print(len(pitch_frequency_data), len(all_keys))
                        print((key_sig, bass_note))
                    #                 all_corpus_labels.append((the_key[0].sharps, bass_note))

            elif type(x) is stream.Opus:
                print('we have an Opus!')
                for y in x:
                    song_chords = []
                    crd = y.chordify().flat.getElementsByClass(chord.Chord)
                    if len(x.flat.getElementsByClass(key.KeySignature)) > 0:
                        key_sig = y.flat.getElementsByClass(key.KeySignature)[0].sharps
                        bass_note = crd[-1].bass().pitchClass
                        modal_type = []
                        if modality == 'tonal':
                            for c in tonal_major_dict.keys():
                                modal_type.append(c)
                            for c in tonal_minor_dict.keys():
                                modal_type.append(c)
                                #                     modal_type.append(c for c in tonal_minor_dict.keys())
                        elif modality == 'modal':
                            for c in modal_major:
                                modal_type.append(c)
                            for c in modal_minor:
                                modal_type.append(c)
                        if (key_sig, bass_note) in modal_type:
                            print((key_sig, bass_note), 'is in there')
                            for c in crd:
                                song_chord = []
                                note_bass = (c.bass().pitchClass - bass_note) % 12
                                song_chord.append(note_bass)
                                song_chord.append(sorted(set([(n.pitchClass - bass_note - note_bass) % 12 for n in c])))
                                song_chords.append(song_chord)
                            all_chords.append(song_chords)
                            all_keys.append((key_sig, bass_note))
                        # this is for pitch class frequency data
                            all_notes = []
                            note_data = {}
                            song_percents = []
                            note_counter = 0
                            for xx in crd:
                                for yy in xx:
                                    all_notes.append((yy.pitchClass - bass_note) % 12)
                                    note_counter += 1
                            for xx in range(12):
                                note_data[xx] = 0
                            for xx in all_notes:
                                note_data[xx] += 1
                            for xx in range(12):
                                song_percents.append(note_data[xx] / len(all_notes) * 100)
                            pitch_frequency_data.append(song_percents)
                            print(len(pitch_frequency_data), len(all_keys))

    start = time.time()
    if type(unparsed) is str:

        for root, dirs, files in os.walk(unparsed):
            for file_name, x in zip(files, range(len(files))):
#                 print(file_name, '...', x + 1, 'of', len(files)+1, '...')
                if file_name.endswith('.krn'):
                    #                     print('it is a krn')
                    path = os.path.join(root, file_name)
                    st = time.time()
                    print('parsing', file_name)
                    all_parsed.append(converter.parse(path))
#                     parser(path)
                    fin = time.time()
                    print('parsed', len(all_parsed), 'of', len(unparsed), 'in', fin - st, 'seconds')
    else:
        for x in unparsed:
            st = time.time()
#             print('parsing', x)
            all_parsed.append(converter.parse(x))
#             parser(x)
            fin = time.time()
            print('parsed', len(all_parsed), 'of', len(unparsed), 'in', fin - st, 'seconds')
    print('entire process took:', time.time() - start, 'seconds')
    parser(all_parsed)
    joblib.dump((all_chords, all_keys), 'pickles/%s_continuo.pkl' % corpus_name, compress=9)
    joblib.dump((pitch_frequency_data, all_keys), 'pickles/%s_notes.pkl' % corpus_name, compress=9)

    print('ALL DONE!!! :) :) :)')

In [ ]:
#crd_data(path_to_folder, mode, corpus_name)
#for mode: If songs composed before 1700, use 'modal', otherwise, use 'tonal'. 
#for corpus_name: name it something simple that is similar to the name of the folder.

data_converter(A, 'modal', 'monteverdi')

#this will print out 'finished 1 of 20...'. It's possible to see "finished 56 out of 20". This is normal; some files
    #contain more than one song. 

#nothing will be saved until this is finished. It should print "ALL DONE!" at the very end unless something went wrong.
#it may take hours, so these are best run overnight

parsed 1 of 97 in 1.488415002822876 seconds
parsed 2 of 97 in 11.710393190383911 seconds
parsed 3 of 97 in 1.2722439765930176 seconds
parsed 4 of 97 in 6.344343900680542 seconds
parsed 5 of 97 in 1.4849927425384521 seconds
parsed 6 of 97 in 7.557887077331543 seconds
parsed 7 of 97 in 2.0217461585998535 seconds
parsed 8 of 97 in 13.627753973007202 seconds
parsed 9 of 97 in 1.763542890548706 seconds
parsed 10 of 97 in 7.690062999725342 seconds
parsed 11 of 97 in 1.1763591766357422 seconds
parsed 12 of 97 in 7.4613940715789795 seconds
parsed 13 of 97 in 0.7616748809814453 seconds
parsed 14 of 97 in 4.444546937942505 seconds
parsed 15 of 97 in 1.2413420677185059 seconds
parsed 16 of 97 in 9.273782014846802 seconds
parsed 17 of 97 in 2.5262868404388428 seconds
parsed 18 of 97 in 5.139539003372192 seconds
parsed 19 of 97 in 1.3440728187561035 seconds
parsed 20 of 97 in 10.959871053695679 seconds
parsed 21 of 97 in 1.5114281177520752 seconds
parsed 22 of 97 in 12.72885513305664 seconds
parsed

In [ ]:
#do it in parallel
import ipyparallel as ipp


rc = ipp.Client()
dview = rc[:]
print(dview)

dview.execute('from music21 import *')
dview.execute('import pickle, time, os')
dview.execute("from sklearn.externals import joblib")

@dview.parallel(block=True)
def parallel_data_converter(number):
    corpus_dict = {0: {'name': 'Zma', 'path': 'kern_files/Zma/', 'modality': 'modal'}, 
                   1: {'name': 'wtc', 'path': 'kern_files/wtc/', 'modality': 'tonal'}}
    tonal_major_dict = {(0, 0): 'CM', (1, 7): 'GM', (2, 2): 'DM', (3, 9): 'AM', (4, 4): 'EM', (5, 11): 'BM', (6, 6): 'F♯M',
               (-1, 5): 'FM', (-2, 10): 'B♭M', (-3, 3): 'E♭M', (-4, 8): 'A♭M', (-5, 1): 'D♭M', (-6, 6): 'G♭M'}
    tonal_minor_dict = {(0, 9): 'am', (1, 4): 'em', (2, 11): 'bm', (3, 6): 'f♯m', (4, 1): 'c♯m', (5, 8): 'g♯m',
                        (6, 3): 'd♯m', (-1, 2): 'dm', (-2, 7): 'gm', (-3, 0): 'cm', (-4, 5): 'fm', (-5, 10): 'b♭m',
                        (-6, 3): 'e♭m'}
    modal_major = [('f', 5), ('n', 0), ('f', 0), ('n', 7), ('f', 10), ('n', 5),
                   (-1, 5), (0, 0), (-1, 0), (0, 7), (-1, 10), (0, 5)]
    modal_minor = [('n', 2), ('n', 9), ('f', 2), ('f', 7), ('n', 4),
                   (0, 2), (0, 9), (-1, 2), (-1, 7), (0, 4)]
    # this function converts .krn or .xml files into the format we need
    def data_converter(unparsed, modality, corpus_name):
        pitch_frequency_data = []
        all_parsed = []
        all_keys = []
        all_chords = []

        def parser(parsed_songs):
            for x in all_parsed:
                if type(x) is stream.Score:
        #             print('we have a score!')
                    song_chords = []
                    crd = x.chordify().flat.getElementsByClass(chord.Chord)
                    if len(x.flat.getElementsByClass(key.KeySignature)) > 0:
                        key_sig = x.flat.getElementsByClass(key.KeySignature)[0].sharps
                        bass_note = crd[-1].bass().pitchClass
                        modal_type = []
                        if modality == 'tonal':
                            for c in tonal_major_dict.keys():
                                modal_type.append(c)
                            for c in tonal_minor_dict.keys():
                                modal_type.append(c)
                                #                 modal_type.append(c for c in tonal_minor_dict.keys())
                        elif modality == 'modal':
                            for c in modal_major:
                                modal_type.append(c)
                            for c in modal_minor:
                                modal_type.append(c)
                                #             print(modal_type, (key_sig, bass_note))
                        if (key_sig, bass_note) in modal_type:
                            print((key_sig, bass_note), 'is in there')
                            for c in crd:
                                song_chord = []
                                note_bass = (c.bass().pitchClass - bass_note) % 12
                                song_chord.append(note_bass)
                                song_chord.append(sorted(set([(n.pitchClass - bass_note - note_bass) % 12 for n in c])))
                                song_chords.append(song_chord)
                            all_chords.append(song_chords)
                            all_keys.append((key_sig, bass_note))
                        # this is for pitch class frequency data
                            all_notes = []
                            note_data = {}
                            song_percents = []
                            note_counter = 0
                            for xx in crd:
                                for yy in xx:
                                    all_notes.append((yy.pitchClass - bass_note) % 12)
                                    note_counter += 1
                            for xx in range(12):
                                note_data[xx] = 0
                            for xx in all_notes:
                                note_data[xx] += 1
                            for xx in range(12):
                                song_percents.append(note_data[xx] / len(all_notes) * 100)
                            pitch_frequency_data.append(song_percents)
                            print(len(pitch_frequency_data), len(all_keys))
                            print((key_sig, bass_note))
                        #                 all_corpus_labels.append((the_key[0].sharps, bass_note))

                elif type(x) is stream.Opus:
                    print('we have an Opus!')
                    for y in x:
                        song_chords = []
                        crd = y.chordify().flat.getElementsByClass(chord.Chord)
                        if len(x.flat.getElementsByClass(key.KeySignature)) > 0:
                            key_sig = y.flat.getElementsByClass(key.KeySignature)[0].sharps
                            bass_note = crd[-1].bass().pitchClass
                            modal_type = []
                            if modality == 'tonal':
                                for c in tonal_major_dict.keys():
                                    modal_type.append(c)
                                for c in tonal_minor_dict.keys():
                                    modal_type.append(c)
                                    #                     modal_type.append(c for c in tonal_minor_dict.keys())
                            elif modality == 'modal':
                                for c in modal_major:
                                    modal_type.append(c)
                                for c in modal_minor:
                                    modal_type.append(c)
                            if (key_sig, bass_note) in modal_type:
                                print((key_sig, bass_note), 'is in there')
                                for c in crd:
                                    song_chord = []
                                    note_bass = (c.bass().pitchClass - bass_note) % 12
                                    song_chord.append(note_bass)
                                    song_chord.append(sorted(set([(n.pitchClass - bass_note - note_bass) % 12 for n in c])))
                                    song_chords.append(song_chord)
                                all_chords.append(song_chords)
                                all_keys.append((key_sig, bass_note))
                            # this is for pitch class frequency data
                                all_notes = []
                                note_data = {}
                                song_percents = []
                                note_counter = 0
                                for xx in crd:
                                    for yy in xx:
                                        all_notes.append((yy.pitchClass - bass_note) % 12)
                                        note_counter += 1
                                for xx in range(12):
                                    note_data[xx] = 0
                                for xx in all_notes:
                                    note_data[xx] += 1
                                for xx in range(12):
                                    song_percents.append(note_data[xx] / len(all_notes) * 100)
                                pitch_frequency_data.append(song_percents)
                                print(len(pitch_frequency_data), len(all_keys))

        start = time.time()
        if type(unparsed) is str:

            for root, dirs, files in os.walk(unparsed):
                for file_name, x in zip(files, range(len(files))):
    #                 print(file_name, '...', x + 1, 'of', len(files)+1, '...')
                    if file_name.endswith('.krn'):
                        #                     print('it is a krn')
                        path = os.path.join(root, file_name)
                        st = time.time()
                        all_parsed.append(converter.parse(path))
    #                     parser(path)
                        fin = time.time()
    #                     print('finished', len(all_chords), 'of', len(unparsed), 'in', fin - st, 'seconds')
        else:
            for x in unparsed:
                st = time.time()
                parsed.append(converter.parse(x))
    #             parser(x)
                fin = time.time()
    #             print('finished', len(all_chords), 'of', len(unparsed), 'in', fin - st, 'seconds')
        print('entire process took:', time.time() - start, 'seconds')
        parser(all_parsed)
        joblib.dump((all_chords, all_keys), 'pickles/%s_continuo.pkl' % corpus_name, compress=9)
        joblib.dump((pitch_frequency_data, all_keys), 'pickles/%s_notes.pkl' % corpus_name, compress=9)

        print('ALL DONE!!! :) :) :)')
    
    data_converter(corpus_dict[number]['path'], corpus_dict[number]['modality'], corpus_dict[number]['name'])

In [ ]:
parallel_data_converter.map(range(2))